In [182]:
import mne
from pathlib import Path
import pandas as pd
import glob
import numpy as np
from tqdm.auto import tqdm

SLEEP_EDF_PATH = Path("./../../data/sleepEDF")
RAW_DATASET_PATH = SLEEP_EDF_PATH / "raw"
RAW_CSV_PATH = SLEEP_EDF_PATH / "raw_csv"

In [304]:
def get_raw_data_info(save=False):
    all_files = glob.glob(f"{RAW_DATASET_PATH}/*")
    all_files = [file.split("/")[-1] for file in all_files]

    hypnogram_files = [file for file in all_files if "Hypnogram" in file]
    psg_files = [file for file in all_files if "PSG" in file]

    info = []
    for hypnogram, psg in zip(hypnogram_files, psg_files):
        subject, night = hypnogram[3:5], hypnogram[5]
        info.append({
            'subject': subject,
            'night': night,
            'hypnogram': hypnogram,
            'psg': psg,
            'raw_csv_path': f"{subject}/{night}.csv"
        })
    info = pd.DataFrame(info)
    
    if save:
        info.to_csv(SLEEP_EDF_PATH / "raw_info.csv", index=False)
        
    return info


def parse_edf_to_epochs_dataframe(data_file, header_file, epoch_duration=30):
    data_file, header_file = RAW_DATASET_PATH / data_file, RAW_DATASET_PATH / header_file

    raw_data = mne.io.read_raw_edf(data_file, stim_channel='Event marker', misc=['Temp rectal'])
    fs = raw_data.info['sfreq']
    labels = mne.read_annotations(header_file)

    event_id = {
        'Sleep stage W': 0,
        'Sleep stage 1': 1,
        'Sleep stage 2': 2,
        'Sleep stage 3': 3,
        'Sleep stage 4': 4,
        'Sleep stage R': 5,
        'Sleep stage M': 6,
    }

    # keep last 30-min wake events before sleep and first 30-min wake events after
    # sleep and redefine annotations on raw data
    labels.crop(labels[1]['onset'] - 30 * 60, labels[-2]['onset'] + 30 * 60)
    raw_data.set_annotations(labels, emit_warning=False)

    events, _ = mne.events_from_annotations(raw_data, event_id=event_id, chunk_duration=epoch_duration)

    tmax = epoch_duration - 1. / fs  # tmax in included

    epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')

    df = epochs.to_data_frame(time_format=None).set_index('epoch')
    
    # make csv smaller 
    df.loc[:, ['condition']] = np.array([event_id[condition] for condition in df['condition'].values]) # encode labels
    # drop time since we know fs and epoch length
    # only EEG data is needed
    df = df[['condition', 'EEG Fpz-Cz', 'EEG Pz-Oz']]    
    return df


def create_sleep_edf_raw_csv_dataset(epoch_duration=30):
    info = get_files_info()
    for row in tqdm(info.iterrows(), total=len(info)):
        row = row[1]
        data_file, header_file, subject, night = row[['psg', 'hypnogram', 'subject', 'night']]
        epochs_df = parse_edf_to_epochs_dataframe(data_file, header_file, epoch_duration=epoch_duration)
        epochs_df.to_csv(RAW_CSV_PATH / f"{subject}_{night}.csv")

In [315]:
all_files = glob.glob(f"{RAW_DATASET_PATH}/*")
all_files = [file.split("/")[-1] for file in all_files]

hypnogram_files = [file for file in all_files if "Hypnogram" in file]
psg_files = [file for file in all_files if "PSG" in file]


---
# **TODO: hypnogramy i psg sa zle posortowane -> zmienic get_info na gorze tak zeby bylo posortowane wzgledem recordsow**
---

In [320]:
np.array(hypnogram_files)[:10], np.array(psg_files)[:10]

(array(['SC4212EC-Hypnogram.edf', 'SC4591GY-Hypnogram.edf',
        'SC4702EA-Hypnogram.edf', 'SC4021EH-Hypnogram.edf',
        'SC4062EC-Hypnogram.edf', 'SC4761EP-Hypnogram.edf',
        'SC4072EH-Hypnogram.edf', 'SC4371FA-Hypnogram.edf',
        'SC4562FJ-Hypnogram.edf', 'SC4721EC-Hypnogram.edf'], dtype='<U22'),
 array(['SC4802G0-PSG.edf', 'SC4022E0-PSG.edf', 'SC4662E0-PSG.edf',
        'SC4442E0-PSG.edf', 'SC4602E0-PSG.edf', 'SC4702E0-PSG.edf',
        'SC4211E0-PSG.edf', 'SC4262F0-PSG.edf', 'SC4021E0-PSG.edf',
        'SC4412E0-PSG.edf'], dtype='<U16'))

In [240]:
data_file, header_file = "SC4001E0-PSG.edf", "SC4001EC-Hypnogram.edf"
df = parse_edf_to_epochs_dataframe(data_file, header_file)

Extracting EDF parameters from /home/shate/Desktop/praca_magisterska/medical_signal_representation/data/sleepEDF/raw/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
841 matching events found
No baseline correction applied
0 projection items activated
Loading data for 841 events and 3000 original time points ...


/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage M (event id 6)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')


0 bad epochs dropped


In [242]:
for row in tqdm(info.iterrows(), total=len(info)):
    row = row[1]
    data_file, header_file, subject, night = row[['psg', 'hypnogram', 'subject', 'night']]
    epochs_df = parse_edf_to_epochs_dataframe(data_file, header_file, epoch_duration=30)
    epochs_df.to_csv(RAW_CSV_PATH / f"{subject}_{night}.csv")

  0%|          | 0/153 [00:00<?, ?it/s]

Extracting EDF parameters from /home/shate/Desktop/praca_magisterska/medical_signal_representation/data/sleepEDF/raw/SC4802G0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
808 matching events found
No baseline correction applied
0 projection items activated
Loading data for 808 events and 3000 original time points ...


/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage 4 (event id 4)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')
/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage M (event id 6)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')


0 bad epochs dropped
Extracting EDF parameters from /home/shate/Desktop/praca_magisterska/medical_signal_representation/data/sleepEDF/raw/SC4022E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1840 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1840 events and 3000 original time points ...


/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage 4 (event id 4)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')
/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage M (event id 6)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')


...

0 bad epochs dropped
Extracting EDF parameters from /home/shate/Desktop/praca_magisterska/medical_signal_representation/data/sleepEDF/raw/SC4041E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1072 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1072 events and 3000 original time points ...


/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage M (event id 6)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')


0 bad epochs dropped
Extracting EDF parameters from /home/shate/Desktop/praca_magisterska/medical_signal_representation/data/sleepEDF/raw/SC4821G0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1054 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1054 events and 3000 original time points ...


/tmp/ipykernel_14361/442409635.py:50: RuntimeWarning: No matching events found for Sleep stage M (event id 6)
  epochs = mne.Epochs(raw=raw_data, events=events, event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='warn')


0 bad epochs dropped


In [307]:
from sklearn.model_selection import GroupShuffleSplit

def create_train_val_test_split_info(random_state=42, save=False):
    info = pd.read_csv(SLEEP_EDF_PATH / "raw_info.csv")

    groups = info['subject'].values
    gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
    train_idx, val_test_idx = next(gss.split(info, groups=groups))

    train_info = info.iloc[train_idx]
    val_test_info = info.iloc[val_test_idx]

    val_test_groups = val_test_info['subject'].values
    gss = GroupShuffleSplit(n_splits=1, train_size=.5, random_state=42)
    val_idx, test_idx = next(gss.split(val_test_info, groups=val_test_groups))

    val_info = info.iloc[val_idx]
    test_info = info.iloc[test_idx]

    train_info.loc[:, 'split'] = 'train'
    val_info.loc[:, 'split'] = 'val'
    test_info.loc[:, 'split'] = 'test'

    train_val_test_split_info = pd.concat([train_info, val_info, test_info])
    
    if save:
        train_val_test_split_info.to_csv(SLEEP_EDF_PATH / "split_info.csv", index=False)
    return train_val_test_split_info

In [314]:
info.set_index('subject').loc[70]

,night,hypnogram,psg,raw_csv_path
subject,,,,
70,2,SC4702EA-Hypnogram.edf,SC4662E0-PSG.edf,70/2.csv
70,1,SC4701EC-Hypnogram.edf,SC4182E0-PSG.edf,70/1.csv


In [308]:
train_val_test_split_info = create_train_val_test_split_info(save=True)

/home/shate/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [303]:
train_val_test_split_info

,subject,night,hypnogram,psg,raw_csv_path,split
0,21,2,SC4212EC-Hypnogram.edf,SC4802G0-PSG.edf,21/2.csv,train
3,2,1,SC4021EH-Hypnogram.edf,SC4442E0-PSG.edf,02/1.csv,train
4,6,2,SC4062EC-Hypnogram.edf,SC4602E0-PSG.edf,06/2.csv,train
5,76,1,SC4761EP-Hypnogram.edf,SC4702E0-PSG.edf,76/1.csv,train
6,7,2,SC4072EH-Hypnogram.edf,SC4211E0-PSG.edf,07/2.csv,train
...,...,...,...,...,...,...
43,44,1,SC4441EC-Hypnogram.edf,SC4111E0-PSG.edf,44/1.csv,test
44,54,2,SC4542FW-Hypnogram.edf,SC4252E0-PSG.edf,54/2.csv,test
45,74,2,SC4742EC-Hypnogram.edf,SC4801G0-PSG.edf,74/2.csv,test
46,38,1,SC4381FC-Hypnogram.edf,SC4651E0-PSG.edf,38/1.csv,test


In [299]:
val_info

,subject,night,hypnogram,psg,raw_csv_path,split
2,70,2,SC4702EA-Hypnogram.edf,SC4662E0-PSG.edf,70/2.csv,val
3,2,1,SC4021EH-Hypnogram.edf,SC4442E0-PSG.edf,02/1.csv,val
6,7,2,SC4072EH-Hypnogram.edf,SC4211E0-PSG.edf,07/2.csv,val
7,37,1,SC4371FA-Hypnogram.edf,SC4262F0-PSG.edf,37/1.csv,val
9,72,1,SC4721EC-Hypnogram.edf,SC4412E0-PSG.edf,72/1.csv,val
14,82,2,SC4822GC-Hypnogram.edf,SC4371F0-PSG.edf,82/2.csv,val
15,1,1,SC4011EH-Hypnogram.edf,SC4591G0-PSG.edf,01/1.csv,val
18,20,2,SC4202EC-Hypnogram.edf,SC4502E0-PSG.edf,20/2.csv,val
20,24,2,SC4242EA-Hypnogram.edf,SC4251E0-PSG.edf,24/2.csv,val
21,77,1,SC4771GC-Hypnogram.edf,SC4141E0-PSG.edf,77/1.csv,val


,subject,night,hypnogram,psg,raw_csv_path,split
0,21,2,SC4212EC-Hypnogram.edf,SC4802G0-PSG.edf,21/2.csv,train
3,2,1,SC4021EH-Hypnogram.edf,SC4442E0-PSG.edf,02/1.csv,train
4,6,2,SC4062EC-Hypnogram.edf,SC4602E0-PSG.edf,06/2.csv,train
5,76,1,SC4761EP-Hypnogram.edf,SC4702E0-PSG.edf,76/1.csv,train
6,7,2,SC4072EH-Hypnogram.edf,SC4211E0-PSG.edf,07/2.csv,train
...,...,...,...,...,...,...
43,44,1,SC4441EC-Hypnogram.edf,SC4111E0-PSG.edf,44/1.csv,test
44,54,2,SC4542FW-Hypnogram.edf,SC4252E0-PSG.edf,54/2.csv,test
45,74,2,SC4742EC-Hypnogram.edf,SC4801G0-PSG.edf,74/2.csv,test
46,38,1,SC4381FC-Hypnogram.edf,SC4651E0-PSG.edf,38/1.csv,test


In [280]:
train_idx.__len__(), val_test_idx.__len__()

(105, 48)

In [282]:
105 / len(info)

0.6862745098039216

In [267]:
gss.get_n_splits()


2